# Access Control Frameowrk

SF is based on:
### DAC - Discretionary access control 

Each object has an owner, who can in turn grant access to that object.

### RBAC - Role-Based Access Control

Access privileges are assigned to roles, which are in turn assigned to users

### UBAC - User-based Access Control :
Only considered when executing: Use secondary role is set to all.

USE SECONDARY ROLES ALL;

Still grants the ownershipt to the primary role.


# Roles

## System-defined Roles:

- GLOBALORGADMIN: organization-level tasks - only in organization accounts.
- ORGADMIN: Phase-out - use GLOBALORGADMIN instead
- ACCOUNTADMIN: encapsulates the SYSADMIN and SECURITYADMIN system-defined roles
- SECURITYADMIN: Can manage any object grant globally, as well as create, monitor, and manage users and roles
- USERADMIN: dedicated to user and role management only
- SYSADMIN: create warehouses and databases (and other objects) in an account
- PUBLIC: Pseudo-role that is automatically granted to every user and every role in your account

System-defined roles cannot be dropped. In addition, the privileges granted to these roles by Snowflake cannot be revoked.


In [ ]:
%%sql -r dataframe_1
use role sysadmin;
create role test;

In [ ]:
%%sql -r dataframe_2
-- to create a role, min requirment is securityadmin

use role securityadmin;
create role test;

In [ ]:
%%sql -r dataframe_3
use role securityadmin;
drop role test;
SHOW ROLES;

In [ ]:
%%sql -r dataframe_4
-- Set up the marketing database
USE ROLE SYSADMIN;

CREATE DATABASE MARKETING;
CREATE SCHEMA SALES;
CREATE TABLE MARKETING.SALES.CAMPAIGN (ID INT, COST NUMERIC,SALES_AMOUNT NUMERIC);


In [ ]:
%%sql -r dataframe_5
-- Use USERADMIN or SECURITYADMIN to set up role
USE ROLE SECURITYADMIN;

-- Create role and user
CREATE ROLE MARKETING_ADMIN;

CREATE USER INITIAL_USER
  PASSWORD = 'AbC201§#';

-- Assign user  
GRANT ROLE MARKETING_ADMIN TO USER INITIAL_USER;

-- Assign role to SYSADMIN -> important to keep the role hierarchy
GRANT ROLE MARKETING_ADMIN TO ROLE SYSADMIN;


## Object Hierarchy

- to grant to objects, you have to also grant usage to the higher objects.
    - ie: granting select access to a table, you are required to grant usage in database and schema(higher objects)

![image info](https://docs.snowflake.com/en/_images/securable-objects-hierarchy.png)



## Role hierachy

Snowflake recommends creating a hierarchy of custom roles, with the top-most custom role assigned to the system role SYSADMIN.

***if a custom role is not assigned to SYSADMIN through a role hierarchy, the system administrators cannot manage the objects owned by the role***

![image info](https://docs.snowflake.com/en/_images/primary-secondary-roles-operations.png)

In [ ]:
%%sql -r dataframe_6
-- Grant privileges

GRANT USAGE ON DATABASE MARKETING TO ROLE MARKETING_ADMIN;
GRANT USAGE ON SCHEMA MARKETING.SALES TO ROLE MARKETING_ADMIN;
GRANT SELECT ON TABLE  MARKETING.SALES.CAMPAIGN TO ROLE MARKETING_ADMIN;
GRANT INSERT ON TABLE  MARKETING.SALES.CAMPAIGN TO ROLE MARKETING_ADMIN;


In [ ]:
%%sql -r dataframe_8
use role sysadmin;
-- Assign warehouse can be done with sysadmin, since sysadmin is the owner of the warehouse
GRANT USAGE ON WAREHOUSE COMPUTE_WH TO ROLE MARKETING_ADMIN;

show grants on warehouse COMPUTE_WH ;

In [ ]:
%%sql -r dataframe_7
use role sysadmin;
-- Create tables
GRANT CREATE TABLE ON SCHEMA MARKETING.SALES TO ROLE MARKETING_ADMIN;

-- -- SELECT on all tables
GRANT SELECT ON ALL TABLES IN SCHEMA  MARKETING.SALES TO ROLE MARKETING_ADMIN;

-- -- SELECT on all future tables
GRANT SELECT ON FUTURE TABLES IN SCHEMA MARKETING.SALES TO ROLE MARKETING_ADMIN;


In [ ]:
%%sql -r dataframe_9
-- What are the privileges of the role?
SHOW GRANTS TO ROLE MARKETING_ADMIN;



In [ ]:
%%sql -r dataframe_10
-- To whom was the role assigned?
SHOW GRANTS OF ROLE MARKETING_ADMIN;


In [ ]:
%%sql -r dataframe_11
-- Drop USER and Database
USE ROLE SYSADMIN;
DROP DATABASE MARKETING;

In [ ]:
%%sql -r dataframe_12

USE ROLE SECURITYADMIN;
DROP ROLE MARKETING_ADMIN;
DROP USER INITIAL_USER;